<a href="https://colab.research.google.com/github/aayushchourasia123/Hackathon-Project/blob/main/Fraud%20Detection%20Using%20Random%20Forest/fraud_detection_model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [24]:
df=pd.read_csv('/content/AIML Dataset.csv')

In [25]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [26]:
df["balanceDiffOrg"]=df["oldbalanceOrg"]-df["newbalanceOrig"]
df["balanceDiffDest"]=df["newbalanceDest"]-df["oldbalanceDest"]

In [27]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,balanceDiffOrg,balanceDiffDest
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0,9839.64,0.0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0,1864.28,0.0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0,181.00,0.0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0,181.00,-21182.0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0,11668.14,0.0


In [28]:
x=df.drop(columns='isFraud')
y=df['isFraud']

In [29]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,stratify=y,random_state=42)

In [30]:
from sklearn.tree import DecisionTreeClassifier

# Re-create the preprocessor to include the new features

# Preprocessor (no scaling for trees)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', ['amount', 'oldbalanceOrg', 'newbalanceOrig',
                                'oldbalanceDest', 'newbalanceDest',
                                'balanceDiffOrg', 'balanceDiffDest']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['type'])
    ],
    remainder='drop'
)


In [31]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, class_weight="balanced", random_state=42),
    "Decision Tree": DecisionTreeClassifier(class_weight="balanced",random_state=42),
    "Random Forest": RandomForestClassifier(class_weight="balanced",random_state=42),
    "XGBoost": XGBClassifier(scale_pos_weight=(len(y[y==0]) / len(y[y==1])),
                              use_label_encoder=False, eval_metric="logloss")
}

In [32]:
results=[]
for name, model in models.items():
    pipe = Pipeline(steps=[("preprocessor", preprocessor),
                           ("classifier", model)])
    pipe.fit(x_train, y_train)
    y_pred = pipe.predict(x_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    cm = confusion_matrix(y_test, y_pred)

    results.append((name, acc, prec, rec, f1, cm))

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [14:17:30] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [33]:
results


[('Logistic Regression',
  0.9490744378887942,
  0.023075529065904193,
  0.9301948051948052,
  0.04503389330975538,
  array([[1809288,   97034],
         [    172,    2292]])),
 ('Decision Tree',
  0.999704524236871,
  0.8971571906354515,
  0.8709415584415584,
  0.8838550247116969,
  array([[1906076,     246],
         [    318,    2146]])),
 ('Random Forest',
  0.9997097631688413,
  0.9626937984496124,
  0.8064123376623377,
  0.8776501766784452,
  array([[1906245,      77],
         [    477,    1987]])),
 ('XGBoost',
  0.9987180333468498,
  0.5017741598831141,
  0.9756493506493507,
  0.6627153687112336,
  array([[1903935,    2387],
         [     60,    2404]]))]

In [34]:
for name, acc, prec, rec, f1, cm in results:
    print("="*50)
    print(f"Model: {name}")
    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1-score: {f1:.4f}")
    print("Confusion Matrix:\n", cm)


Model: Logistic Regression
Accuracy: 0.9491
Precision: 0.0231
Recall: 0.9302
F1-score: 0.0450
Confusion Matrix:
 [[1809288   97034]
 [    172    2292]]
Model: Decision Tree
Accuracy: 0.9997
Precision: 0.8972
Recall: 0.8709
F1-score: 0.8839
Confusion Matrix:
 [[1906076     246]
 [    318    2146]]
Model: Random Forest
Accuracy: 0.9997
Precision: 0.9627
Recall: 0.8064
F1-score: 0.8777
Confusion Matrix:
 [[1906245      77]
 [    477    1987]]
Model: XGBoost
Accuracy: 0.9987
Precision: 0.5018
Recall: 0.9756
F1-score: 0.6627
Confusion Matrix:
 [[1903935    2387]
 [     60    2404]]


In [42]:
#choosing random forest
rf_pipeline = Pipeline(steps=[("preprocessor", preprocessor),
                           ("classifier", RandomForestClassifier(class_weight="balanced", random_state=42,n_jobs=-1)
)])

In [43]:
rf_pipeline.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['amount', 'oldbalanceOrg',
                                                   'newbalanceOrig',
                                                   'oldbalanceDest',
                                                   'newbalanceDest',
                                                   'balanceDiffOrg',
                                                   'balanceDiffDest']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['type'])])),
                ('classifier',
                 RandomForestClassifier(class_weight='balanced', n_jobs=-1,
                                        random_state=42))])

In [44]:
y_pred_rf=rf_pipeline.predict(x_test)

In [45]:
print(classification_report(y_test,y_pred_rf))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1906322
           1       0.96      0.81      0.88      2464

    accuracy                           1.00   1908786
   macro avg       0.98      0.90      0.94   1908786
weighted avg       1.00      1.00      1.00   1908786



In [46]:
print(confusion_matrix(y_test,y_pred_rf))

[[1906245      77]
 [    477    1987]]


In [47]:
rf_pipeline.score(x_test,y_test)*100

99.97097631688413

In [48]:
import joblib
joblib.dump(rf_pipeline, "fraud_detection_random_forest_pipeline.pkl")

['fraud_detection_random_forest_pipeline.pkl']